<a href="https://colab.research.google.com/github/waleowoeye/gemini-cookbook/blob/5-days-genai/Fine_tuning_a_custom_model_for_security.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2025 Google LLC.

In [ ]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Day 4 - Fine tuning a custom model

Welcome back to the Kaggle 5-day Generative AI course!

In this notebook you will use the Gemini API to fine-tune a custom, task-specific model. Fine-tuning can be used for a variety of tasks from classic NLP problems like entity extraction or summarisation, to creative tasks like stylised generation. You will fine-tune a model to classify the category a piece of text (a newsgroup post) into the category it belongs to (the newsgroup name).

This codelab walks you tuning a model with the API. [AI Studio](https://aistudio.google.com/app/tune) also supports creating new tuned models directly in the web UI, allowing you to quickly create and monitor models using data from Google Sheets, Drive or your own files.

**Note**: We recommend doing this codelab first today. There may be a period of waiting while the model tunes, so if you start with this one, you can try the other codelab while you wait.

In [ ]:
!pip uninstall -qqy jupyterlab  # Remove unused conflicting packages
!pip install -U -q "google-genai==1.7.0"

In [ ]:
from google import genai
from google.genai import types

genai.__version__

'1.7.0'

### Set up your API key

To run the following cell, your API key must be stored it in a [Kaggle secret](https://www.kaggle.com/discussions/product-feedback/114053) named `GOOGLE_API_KEY`.

If you don't already have an API key, you can grab one from [AI Studio](https://aistudio.google.com/app/apikey). You can find [detailed instructions in the docs](https://ai.google.dev/gemini-api/docs/api-key).

To make the key available through Kaggle secrets, choose `Secrets` from the `Add-ons` menu and follow the instructions to add your key or enable it for this notebook.

In [ ]:
from google.colab import userdata
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
client = genai.Client(api_key=GOOGLE_API_KEY)

If you received an error response along the lines of `No user secrets exist for kernel id ...`, then you need to add your API key via `Add-ons`, `Secrets` **and** enable it.

![Screenshot of the checkbox to enable GOOGLE_API_KEY secret](https://storage.googleapis.com/kaggle-media/Images/5gdai_sc_3.png)

### Explore available models

You will be using the [`TunedModel.create`](https://ai.google.dev/api/tuning#method:-tunedmodels.create) API method to start the fine-tuning job and create your custom model. Find a model that supports it through the [`models.list`](https://ai.google.dev/api/models#method:-models.list) endpoint. You can also find more information about tuning models in [the model tuning docs](https://ai.google.dev/gemini-api/docs/model-tuning/tutorial?lang=python).

In [ ]:
for model in client.models.list():
    if "createTunedModel" in model.supported_actions:
        print(model.name)

models/gemini-1.5-flash-001-tuning


In [ ]:
#Uploading the zip file

In [ ]:
import os

# List uploaded files
files = os.listdir('/content/secData')
print(files)
for file in files:
  print("Uploaded File:", file)
'''
secContent = files.upload()
uploaded_files = list(secContent.keys())
print("Uploaded Files:", uploaded_files)
'''

['dns.txt.gz', 'flows.txt.gz', 'redteam.txt.gz']
Uploaded File: dns.txt.gz
Uploaded File: flows.txt.gz
Uploaded File: redteam.txt.gz


'\nsecContent = files.upload()\nuploaded_files = list(secContent.keys())\nprint("Uploaded Files:", uploaded_files)\n'

unzipping it

In [ ]:
import gzip
import shutil
import os

# Make sure your folder exists
os.makedirs("/content/data", exist_ok=True)

#Loop through each file
for file in files:
    # Define the path for the extracted file
    output_file = os.path.join('/content/data', os.path.splitext(file)[0])  # remove .gz extension

    # Check if the file has already been extracted
    if not os.path.exists(output_file):  # Only unzip if it doesn't already exist
        # Path to the original .gz file
        gz_path = os.path.join('/content/secData', file)

        # Check file integrity before attempting to extract
        print(f"Checking integrity of {gz_path}...")  # Added for clarity
        !gzip -t "$gz_path"  # Using shell command to check integrity
        if os.WEXITSTATUS(os.system(f"gzip -t '{gz_path}'")) != 0:
            print(f"File {gz_path} is corrupted. Skipping extraction.")
            continue

        # Unzip the file
        with gzip.open(gz_path, 'rb') as f_in:
            with open(output_file, 'wb') as f_out:
                shutil.copyfileobj(f_in, f_out)

        print(f"Extracted: {output_file}")
    else:
        print(f"File {output_file} already exists, skipping extraction.")

File /content/data/dns.txt already exists, skipping extraction.
Checking integrity of /content/secData/flows.txt.gz...

gzip: /content/secData/flows.txt.gz: unexpected end of file
File /content/secData/flows.txt.gz is corrupted. Skipping extraction.
File /content/data/redteam.txt already exists, skipping extraction.


## Download the dataset

In this activity, we will use the cybersecurity dataset in (https://www.kaggle.com/code/markishere/day-2-classihttps://csr.lanl.gov/data/cyber1/). In this example you will use a fine-tuned Gemini model to achieve the same goal.

The dataset contains 3 files
['/content/data/redteam.txt', '/content/data/dns.txt', '/content/data/flows.txt']

use the name of each file as a label, so
- redteam.txt will be labeled redteam
- dns.txt will be labeled dns and
- flows.txt will be labeled flows

However, I want to combine the 3 files first, then break them into train and test data

Per the data set, with the first line as header
flows.txt will contain
time,duration,source computer,source port,destination computer,destination port,protocol,packet count,byte count
1,9,C3090,N10471,C3420,N46,6,3,144
1,9,C3538,N2600,C3371,N46,6,3,144
2,0,C4316,N10199,C5030,443,6,2,92

dns.txt will contain
time,source computer,computer resolved
31,C161,C2109
35,C5642,C528
38,C3380,C22841

redteam.txt
time,user@domain,source computer,destination computer
151648,U748@DOM1,C17693,C728
151993,U6115@DOM1,C17693,C1173
153792,U636@DOM1,C17693,C294


Here's what a single row looks like.

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

# File paths
files = {
    'redteam': '/content/data/redteam.txt',
    'dns': '/content/data/dns.txt',
    'flows': '/content/data/flows.txt'
}

# Create an empty DataFrame to combine data
combined_data = pd.DataFrame()

# Loop through the files and read each one
for label, file_path in files.items():
    # Read the file into a DataFrame
    df = pd.read_csv(file_path)

    # Add a new column with the label
    df['label'] = label

    # Append to the combined DataFrame
    combined_data = pd.concat([combined_data, df], ignore_index=True)

# Show the combined data
print("Combined Data Preview:")
print(combined_data.head())

# Split the combined data into train and test sets (80% train, 20% test)
train_data, test_data = train_test_split(combined_data, test_size=0.2, random_state=42, stratify=combined_data['label'])

# Show the split data
print("\nTrain Data Preview:")
print(train_data.head())

print("\nTest Data Preview:")
print(test_data.head())


ParserError: Error tokenizing data. C error: Expected 3 fields in line 1071011, saw 6


In [ ]:
print(f"Total files: {len(file_paths)}")
print(f"Training files: {len(train_files)}")
print(f"Test files: {len(test_files)}")

print('*'*50)
for idx, file in enumerate(train_files[:len(train_files)]):
  print(f"Train File {idx + 1}: {file}, Label: {train_labels[idx]}")

  if idx < len(train_files):
    try:
      with open(file, 'r', encoding='utf-8', errors='ignore') as f:
        print(f"\n--- Preview of File {idx + 1} ---")
        print(f.read()[:500])  # First 500 characters
    except Exception as e:
      print(f"Error reading file {file}: {e}")

'''
for i in range(5):  # show first 5 test files
  print(f"Train File: {train_files[i]}, Label: {train_labels[i]}")
  # Show a few lines from some training files
  for i in range(3):
    with open(train_files[i], 'r', encoding='utf-8', errors='ignore') as f:
        print(f"\n--- File: {train_files[i]} ---")
        print(f.read()[:500])  # first 500 characters


  print(f"Test File: {test_files[i]}, Label: {test_labels[i]}")
  # Show a few lines from some test files
  for i in range(3):
    with open(test_files[i], 'r', encoding='utf-8', errors='ignore') as f:
        print(f"\n--- File: {test_files[i]} ---")
        print(f.read()[:500])  # first 500 characters

'''
print('*'*50)
for idx, file in enumerate(test_files[:len(test_files)]):
  print(f"Train File {idx + 1}: {file}, Label: {test_labels[idx]}")

  if idx < len(test_files):
    try:
      with open(file, 'r', encoding='utf-8', errors='ignore') as f:
        print(f"\n--- Preview of File {idx + 1} ---")
        print(f.read()[:500])  # First 500 characters
    except Exception as e:
      print(f"Error reading file {file}: {e}")


Total files: 3
Training files: 2
Test files: 1
**************************************************
Train File 1: /content/data/dns.txt, Label: data

--- Preview of File 1 ---
2,C4653,C5030
2,C5782,C16712
6,C1191,C419
15,C3380,C22841
18,C2436,C5030
31,C161,C2109
35,C5642,C528
38,C3380,C22841
42,C2428,C1065
42,C2428,C2109
42,C2428,C586
43,C2428,C528
44,C2428,C2109
49,C5778,C706
50,C2492,C457
51,C1299,C2109
53,C608,C706
59,C2517,C1707
62,C24436,C24436
62,C26743,C26743
62,C5778,C1685
63,C3198,C1707
63,C5778,C1685
65,C1108,C706
65,C2428,C457
66,C2428,C457
71,C4651,C1707
73,C2428,C1685
76,C18636,C1685
76,C18636,C1707
76,C3380,C22841
80,C2303,C25571
87,C2428,C625
88,C2428,C
Train File 2: /content/data/redteam.txt, Label: data

--- Preview of File 2 ---
150885,U620@DOM1,C17693,C1003
151036,U748@DOM1,C17693,C305
151648,U748@DOM1,C17693,C728
151993,U6115@DOM1,C17693,C1173
153792,U636@DOM1,C17693,C294
155219,U748@DOM1,C17693,C5693
155399,U748@DOM1,C17693,C152
155460,U748@DOM1,C17693,C2341
155591,U

In [ ]:
from collections import Counter

print("Training label distribution:")
print(Counter(train_labels))

print("\nTest label distribution:")
print(Counter(test_labels))

Training label distribution:
Counter({'data': 2})

Test label distribution:
Counter({'data': 1})


## Prepare the dataset

You'll use the same pre-processing code you used for the custom model on day 2. This pre-processing removes personal information, which can be used to "shortcut" to known users of a forum, and formats the text to appear a bit more like regular text and less like a newsgroup post (e.g. by removing the mail headers). This normalisation allows the model to generalise to regular text and not over-depend on specific fields. If your input data is always going to be newsgroup posts, it may be helpful to leave this structure in place if they provide genuine signals.

In [ ]:
import email
import re

import pandas as pd


def preprocess_newsgroup_row(data):
    # Extract only the subject and body
    msg = email.message_from_string(data)
    text = f"{msg['Subject']}\n\n{msg.get_payload()}"
    # Strip any remaining email addresses
    text = re.sub(r"[\w\.-]+@[\w\.-]+", "", text)
    # Truncate the text to fit within the input limits
    text = text[:40000]

    return text


def preprocess_newsgroup_data(newsgroup_dataset):
    # Put data points into dataframe
    df = pd.DataFrame(
        {"Text": newsgroup_dataset.data, "Label": newsgroup_dataset.target}
    )
    # Clean up the text
    df["Text"] = df["Text"].apply(preprocess_newsgroup_row)
    # Match label to target name index
    df["Class Name"] = df["Label"].map(lambda l: newsgroup_dataset.target_names[l])

    return df

In [ ]:
# Apply preprocessing to training and test datasets
df_train = preprocess_newsgroup_data(newsgroups_train)
df_test = preprocess_newsgroup_data(newsgroups_test)

df_train.head()

,Text,Label,Class Name
0,WHAT car is this!?\n\n I was wondering if anyo...,7,rec.autos
1,SI Clock Poll - Final Call\n\nA fair number of...,4,comp.sys.mac.hardware
2,"PB questions...\n\nwell folks, my mac plus fin...",4,comp.sys.mac.hardware
3,Re: Weitek P9000 ?\n\nRobert J.C. Kyanko () wr...,1,comp.graphics
4,Re: Shuttle Launch Question\n\nFrom article <>...,14,sci.space


Now sample the data. You will keep 50 rows for each category for training. Note that this is even fewer than the Keras example, as this technique (parameter-efficient fine-tuning, or PEFT) updates a relatively small number of parameters and does not require training a new model or updating the large model.

In [ ]:
def sample_data(df, num_samples, classes_to_keep):
    # Sample rows, selecting num_samples of each Label.
    df = (
        df.groupby("Label")[df.columns]
        .apply(lambda x: x.sample(num_samples))
        .reset_index(drop=True)
    )

    df = df[df["Class Name"].str.contains(classes_to_keep)]
    df["Class Name"] = df["Class Name"].astype("category")

    return df


TRAIN_NUM_SAMPLES = 50
TEST_NUM_SAMPLES = 10
# Keep rec.* and sci.*
CLASSES_TO_KEEP = "^rec|^sci"

df_train = sample_data(df_train, TRAIN_NUM_SAMPLES, CLASSES_TO_KEEP)
df_test = sample_data(df_test, TEST_NUM_SAMPLES, CLASSES_TO_KEEP)

## Evaluate baseline performance

Before you start tuning a model, it's good practice to perform an evaluation on the available models to ensure you can measure how much the tuning helps.

First identify a single sample row to use for visual inspection.

In [ ]:
sample_idx = 0
sample_row = preprocess_newsgroup_row(newsgroups_test.data[sample_idx])
sample_label = newsgroups_test.target_names[newsgroups_test.target[sample_idx]]

print(sample_row)
print('---')
print('Label:', sample_label)

Need info on 88-89 Bonneville


 I am a little confused on all of the models of the 88-89 bonnevilles.
I have heard of the LE SE LSE SSE SSEI. Could someone tell me the
differences are far as features or performance. I am also curious to
know what the book value is for prefereably the 89 model. And how much
less than book value can you usually get them for. In other words how
much are they in demand this time of year. I have heard that the mid-spring
early summer is the best time to buy.

			Neil Gandler

---
Label: rec.autos


Passing the text directly in as a prompt does not yield the desired results. The model will attempt to respond to the message.

In [ ]:
response = client.models.generate_content(
    model="gemini-1.5-flash-001", contents=sample_row)
print(response.text)

##  Bonneville Models: Deciphering the Acronyms

You're right, the Bonneville model names can get confusing! Here's a breakdown of the 88-89 models and their key differences:

**1. Bonneville:** This was the base model, often referred to as the "standard" Bonneville. It came with a 3.8L V6 engine and a variety of options like air conditioning, power steering, and automatic transmission.

**2. Bonneville LE:** The LE (Luxury Edition) package added features like leather seating, power windows and locks, and a more upscale interior. It was still powered by the 3.8L V6.

**3. Bonneville SE:** The SE (Special Edition) was similar to the LE but often included slightly different trim, interior colors, and wheels. It was also powered by the 3.8L V6.

**4. Bonneville LSE:** The LSE (Luxury Sport Edition) combined features of the LE and SE with a more powerful 3.8L V6 engine and sport-tuned suspension. 

**5. Bonneville SSE:** The SSE (Super Sport Edition) was the top-of-the-line Bonneville. It 

You can use the prompt engineering techniques you have learned this week to induce the model to perform the desired task. Try some of your own ideas and see what is effective, or check out the following cells for different approaches. Note that they have different levels of effectiveness!

In [ ]:
# Ask the model directly in a zero-shot prompt.

prompt = "From what newsgroup does the following message originate?"
baseline_response = client.models.generate_content(
    model="gemini-1.5-flash-001",
    contents=[prompt, sample_row])
print(baseline_response.text)

The message most likely originates from a newsgroup focused on **Buick vehicles**, specifically the **Bonneville model**.

Here's why:

* **Subject:** The message clearly discusses the 1988-1989 Buick Bonneville, a popular car model during that time.
* **Terminology:** The message uses specific Bonneville trim levels like LE, SE, LSE, SSE, and SSEi, indicating a focus on this specific model.
* **Information Requested:**  The author seeks information about the differences between these trim levels and their value. This suggests an interest in buying or owning a Bonneville.

Based on these clues, a likely newsgroup would be:

* **alt.autos.buick:** A newsgroup dedicated to all things Buick.
* **rec.autos.buick:** Another newsgroup focused on Buick cars and related discussions.
* **alt.autos.bonneville:** A more specific newsgroup devoted to the Buick Bonneville model. 

You might find similar discussions on car enthusiast forums or online communities focused on classic cars and Buick mod

This technique still produces quite a verbose response. You could try and parse out the relevant text, or refine the prompt even further.

In [ ]:
from google.api_core import retry

# You can use a system instruction to do more direct prompting, and get a
# more succinct answer.

system_instruct = """
You are a classification service. You will be passed input that represents
a newsgroup post and you must respond with the newsgroup from which the post
originates.
"""

# Define a helper to retry when per-minute quota is reached.
is_retriable = lambda e: (isinstance(e, genai.errors.APIError) and e.code in {429, 503})

# If you want to evaluate your own technique, replace this body of this function
# with your model, prompt and other code and return the predicted answer.
@retry.Retry(predicate=is_retriable)
def predict_label(post: str) -> str:
    response = client.models.generate_content(
        model="gemini-1.5-flash-001",
        config=types.GenerateContentConfig(
            system_instruction=system_instruct),
        contents=post)

    rc = response.candidates[0]

    # Any errors, filters, recitation, etc we can mark as a general error
    if rc.finish_reason.name != "STOP":
        return "(error)"
    else:
        # Clean up the response.
        return response.text.strip()


prediction = predict_label(sample_row)

print(prediction)
print()
print("Correct!" if prediction == sample_label else "Incorrect.")

rec.autos.pontiac

Incorrect.


Now run a short evaluation using the function defined above. The test set is further sampled to ensure the experiment runs smoothly on the API's free tier. In practice you would evaluate over the whole set.

In [ ]:
import tqdm
from tqdm.rich import tqdm as tqdmr
import warnings

# Enable tqdm features on Pandas.
tqdmr.pandas()

# But suppress the experimental warning
warnings.filterwarnings("ignore", category=tqdm.TqdmExperimentalWarning)


# Further sample the test data to be mindful of the free-tier quota.
df_baseline_eval = sample_data(df_test, 2, '.*')

# Make predictions using the sampled data.
df_baseline_eval['Prediction'] = df_baseline_eval['Text'].progress_apply(predict_label)

# And calculate the accuracy.
accuracy = (df_baseline_eval["Class Name"] == df_baseline_eval["Prediction"]).sum() / len(df_baseline_eval)
print(f"Accuracy: {accuracy:.2%}")

Output()

Accuracy: 25.00%


Now take a look at the dataframe to compare the predictions with the labels.

In [ ]:
df_baseline_eval

,Text,Label,Class Name,Prediction
0,Drag Coefficients\n\nCould someone explain how...,7,rec.autos,comp.sys.ibm.pc.hardware
1,Alternative Fuel Vehicles\n\n\nHere is a press...,7,rec.autos,alt.autos.alternative-fuels
2,"Re: Test ride on a GTS1000\n\nIn article , (T...",8,rec.motorcycles,rec.motorcycles
3,Re: Bra... bra... brazing (Was: For )\n\nOld...,8,rec.motorcycles,rec.motorcycles
4,Re: Twins Update 4-22\n\n (Paul R Krueger) wri...,9,rec.sport.baseball,rec.sports.baseball.mlb
5,Re: Jack Morris\n\nIn article <> (Roger Mayna...,9,rec.sport.baseball,rec.sport.baseball
6,Re: ESPN f*ck up\n\nIn article <> (Matt Coohi...,10,rec.sport.hockey,rec.sports.hockey
7,Re: Lemieux's Getting the Hart... Jeez I hope ...,10,rec.sport.hockey,rec.sport.hockey
8,Re: Off the shelf cheap DES keyseach machine (...,11,sci.crypt,(error)
9,"Re: Overreacting (was Re: Once tapped, your co...",11,sci.crypt,alt.politics.clinton


## Tune a custom model

In this example you'll use tuning to create a model that requires no prompting or system instructions and outputs succinct text from the classes you provide in the training data.

The data contains both input text (the processed posts) and output text (the category, or newsgroup), that you can use to start tuning a model.

When calling `tune()`, you can specify model tuning hyperparameters too:
 - `epoch_count`: defines how many times to loop through the data,
 - `batch_size`: defines how many rows to process in a single step, and
 - `learning_rate`: defines the scaling factor for updating model weights at each step.

You can also choose to omit them and use the defaults. [Learn more](https://developers.google.com/machine-learning/crash-course/linear-regression/hyperparameters) about these parameters and how they work. For this example these parameters were selected by running some tuning jobs and selecting parameters that converged efficiently.

This example will start a new tuning job, but only if one does not already exist. This allows you to leave this codelab and come back later - re-running this step will find your last model.

In [ ]:
from collections.abc import Iterable
import random


# Convert the data frame into a dataset suitable for tuning.
input_data = {'examples':
    df_train[['Text', 'Class Name']]
      .rename(columns={'Text': 'textInput', 'Class Name': 'output'})
      .to_dict(orient='records')
 }

# If you are re-running this lab, add your model_id here.
model_id = None

# Or try and find a recent tuning job.
if not model_id:
  queued_model = None
  # Newest models first.
  for m in reversed(client.tunings.list()):
    # Only look at newsgroup classification models.
    if m.name.startswith('tunedModels/newsgroup-classification-model'):
      # If there is a completed model, use the first (newest) one.
      if m.state.name == 'JOB_STATE_SUCCEEDED':
        model_id = m.name
        print('Found existing tuned model to reuse.')
        break

      elif m.state.name == 'JOB_STATE_RUNNING' and not queued_model:
        # If there's a model still queued, remember the most recent one.
        queued_model = m.name
  else:
    if queued_model:
      model_id = queued_model
      print('Found queued model, still waiting.')


# Upload the training data and queue the tuning job.
if not model_id:
    tuning_op = client.tunings.tune(
        base_model="models/gemini-1.5-flash-001-tuning",
        training_dataset=input_data,
        config=types.CreateTuningJobConfig(
            tuned_model_display_name="Newsgroup classification model",
            batch_size=16,
            epoch_count=2,
        ),
    )

    print(tuning_op.state)
    model_id = tuning_op.name

print(model_id)

This has created a tuning job that will run in the background. To inspect the progress of the tuning job, run this cell to plot the current status and loss curve. Once the status reaches `ACTIVE`, tuning is complete and the model is ready to use.

Tuning jobs are queued, so it may look like no training steps have been taken initially but it will progress. Tuning can take anywhere from a few minutes to multiple hours, depending on factors like your dataset size and how busy the tuning infrastrature is. Why not treat yourself to a nice cup of tea while you wait, or come and say "Hi!" in the group [Discord](https://discord.com/invite/kaggle).

It is safe to stop this cell at any point. It will not stop the tuning job.

**IMPORTANT**: Due to the high volume of users doing this course, tuning jobs may be queued for many hours. Take a note of your tuned model ID above (`tunedModels/...`) so you can come back to it tomorrow. In the meantime, check out the [Search grounding](https://www.kaggle.com/code/markishere/day-4-google-search-grounding/) codelab. If you want to try tuning a local LLM, check out [the fine-tuning guides for tuning a Gemma model](https://ai.google.dev/gemma/docs/tune).

In [ ]:
import datetime
import time


MAX_WAIT = datetime.timedelta(minutes=10)

while not (tuned_model := client.tunings.get(name=model_id)).has_ended:

    print(tuned_model.state)
    time.sleep(60)

    # Don't wait too long. Use a public model if this is going to take a while.
    if datetime.datetime.now(datetime.timezone.utc) - tuned_model.create_time > MAX_WAIT:
        print("Taking a shortcut, using a previously prepared model.")
        model_id = "tunedModels/newsgroup-classification-model-ltenbi1b"
        tuned_model = client.tunings.get(name=model_id)
        break


print(f"Done! The model state is: {tuned_model.state.name}")

if not tuned_model.has_succeeded and tuned_model.error:
    print("Error:", tuned_model.error)

## Use the new model

Now that you have a tuned model, try it out with custom data. You use the same API as a normal Gemini API interaction, but you specify your new model as the model name, which will start with `tunedModels/`.

In [ ]:
new_text = """
First-timer looking to get out of here.

Hi, I'm writing about my interest in travelling to the outer limits!

What kind of craft can I buy? What is easiest to access from this 3rd rock?

Let me know how to do that please.
"""

response = client.models.generate_content(
    model=model_id, contents=new_text)

print(response.text)

### Evaluation

You can see that the model outputs labels that correspond to those in the training data, and without any system instructions or prompting, which is already a great improvement. Now see how well it performs on the test set.

Note that there is no parallelism in this example; classifying the test sub-set will take a few minutes.

In [ ]:
@retry.Retry(predicate=is_retriable)
def classify_text(text: str) -> str:
    """Classify the provided text into a known newsgroup."""
    response = client.models.generate_content(
        model=model_id, contents=text)
    rc = response.candidates[0]

    # Any errors, filters, recitation, etc we can mark as a general error
    if rc.finish_reason.name != "STOP":
        return "(error)"
    else:
        return rc.content.parts[0].text


# The sampling here is just to minimise your quota usage. If you can, you should
# evaluate the whole test set with `df_model_eval = df_test.copy()`.
df_model_eval = sample_data(df_test, 4, '.*')

df_model_eval["Prediction"] = df_model_eval["Text"].progress_apply(classify_text)

accuracy = (df_model_eval["Class Name"] == df_model_eval["Prediction"]).sum() / len(df_model_eval)
print(f"Accuracy: {accuracy:.2%}")

## Compare token usage

AI Studio and the Gemini API provide model tuning at no cost, however normal limits and charges apply for *use* of a tuned model.

The size of the input prompt and other generation config like system instructions, as well as the number of generated output tokens, all contribute to the overall cost of a request.

In [ ]:
# Calculate the *input* cost of the baseline model with system instructions.
sysint_tokens = client.models.count_tokens(
    model='gemini-1.5-flash-001', contents=[system_instruct, sample_row]
).total_tokens
print(f'System instructed baseline model: {sysint_tokens} (input)')

# Calculate the input cost of the tuned model.
tuned_tokens = client.models.count_tokens(model=tuned_model.base_model, contents=sample_row).total_tokens
print(f'Tuned model: {tuned_tokens} (input)')

savings = (sysint_tokens - tuned_tokens) / tuned_tokens
print(f'Token savings: {savings:.2%}')  # Note that this is only n=1.

The earlier verbose model also produced more output tokens than needed for this task.

In [ ]:
baseline_token_output = baseline_response.usage_metadata.candidates_token_count
print('Baseline (verbose) output tokens:', baseline_token_output)

tuned_model_output = client.models.generate_content(
    model=model_id, contents=sample_row)
tuned_tokens_output = tuned_model_output.usage_metadata.candidates_token_count
print('Tuned output tokens:', tuned_tokens_output)

## Next steps

Now that you have tuned a classification model, try some other tasks, like tuning a model to respond with a specific tone or style using hand-written examples (or even generated examples!). Kaggle hosts [a number of datasets](https://www.kaggle.com/datasets) you can try out.

Learn about [when supervised fine-tuning is most effective](https://cloud.google.com/blog/products/ai-machine-learning/supervised-fine-tuning-for-gemini-llm).

And check out the [fine-tuning tutorial](https://ai.google.dev/gemini-api/docs/model-tuning/tutorial?hl=en&lang=python) for another example that shows a tuned model extending beyond the training data to new, unseen inputs.

*- [Mark McD](https://linktr.ee/markmcd)*